In [1]:
# importing modules needed

import os
import pandas as pd
import numpy as np
from scipy import stats
import openpyxl
import matplotlib.pyplot as plt
import scipy
from scipy.fftpack import irfft, rfft

%matplotlib inline
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
import seaborn as sns

In [2]:
# folder path created for code and also the raw files
p1 = os.getcwd()
p2 = 'Final_Raw_Data'
path = os.path.join(p1,p2)

In [9]:
df = pd.read_excel('299cfe16.xlsx')

In [11]:
df.columns

Index(['Data_pull_time', 'Mac_Id', 'Axis', 'Vsample1', 'Vsample2', 'Vsample3',
       'Vsample4', 'Vsample5', 'Vsample6', 'Vsample7',
       ...
       'Vsample4087', 'Vsample4088', 'Vsample4089', 'Vsample4090',
       'Vsample4091', 'Vsample4092', 'Vsample4093', 'Vsample4094',
       'Vsample4095', 'Vsample4096'],
      dtype='object', length=4099)

In [14]:
processed = pd.DataFrame()
processed = df[['Data_pull_time', 'Mac_Id', 'Axis']]

In [18]:
processed.head()

,Data_pull_time,Mac_Id,Axis,RMS
0,2020-08-31 08:34:36,29:9c:fe:16,X,17.398653
1,2020-08-31 09:07:45,29:9c:fe:16,X,17.105179
2,2020-08-31 09:32:04,29:9c:fe:16,X,19.647579
3,2020-08-31 11:07:56,29:9c:fe:16,X,23.202746
4,2020-08-31 11:48:18,29:9c:fe:16,X,21.528653


In [17]:
processed['RMS'] =  (df.loc[:,'Vsample1':'Vsample4096']**2).sum(1).pow(1/2)

C:\Users\husssabe\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
Data_final.Defect_Introduced.value_counts()

In [ ]:
Data_final.set_index("index",inplace=True)

In [ ]:
Data_final.reset_index(inplace=True)

In [ ]:
Data_final.head()

In [ ]:

Data = Data_final.copy()
#processed = Data.loc[:,'Data_pull_time':'Sensor_Loc']


processed['Max'] = Data.loc[:,'Vsample1':'Vsample4096'].max(axis=1)
processed['Min'] = Data.loc[:,'Vsample1':'Vsample4096'].min(axis=1)
processed['Mean'] = Data.loc[:,'Vsample1':'Vsample4096'].mean(axis=1)
processed['Std'] = Data.loc[:,'Vsample1':'Vsample4096'].std(axis=1)
processed['P2P'] = Data.loc[:,'Vsample1':'Vsample4096'].max(axis=1) - Data.loc[:,'Vsample1':'Vsample4096'].min(axis=1)
processed['Kurt'] = Data.loc[:,'Vsample1':'Vsample4096'].kurt(axis=1)
processed['RMS'] =  (Data.loc[:,'Vsample1':'Vsample4096']**2).sum(1).pow(1/2)
processed['Var'] = Data.loc[:,'Vsample1':'Vsample4096'].var(axis=1)

In [ ]:
processed.head()
#processed.Defect_Introduced.value_counts()

### adding frequency bins to processed data

In [ ]:
Data_Processed = pd.DataFrame()
sec = 1.2792877225866917
N = 4096
fs_rate=round(N/sec)
fs_rate
Ts = 1.0/fs_rate # sampling interval in time
t = np.arange(0, secs, Ts) # time vector as scipy arange field / numpy.ndarray
freqs = scipy.fftpack.fftfreq(N, t[1]-t[0])
print(len(freqs))
freqs_side = freqs[range(N//2)] # one side frequency range
print(len(freqs_side))

In [ ]:
Data_Processed = pd.DataFrame()
sec = 1.2792877225866917
N = 4096
fs_rate=round(N/sec)
fs_rate
Ts = 1.0/fs_rate # sampling interval in time
t = np.arange(0, secs, Ts) # time vector as scipy arange field / numpy.ndarray
freqs = scipy.fftpack.fftfreq(N, t[1]-t[0])
freqs_side = freqs[range(N//2)] # one side frequency range
## high pass filter
for i,row in Data.iterrows():
    data = row['Vsample1':'Vsample4096'] - row['Vsample1':'Vsample4096'].mean() # subracting mean from the samples
    #print(row["Data_pull_time":"Sensor_Loc"])
    f_signal = rfft(data)
    #print(row['Vsample1':'Vsample6'])
    #print(f_signal[:5])
    #print(len(f_signal))
    #If our original signal time was in seconds, this is now in Hz    
    cut_f_signal = f_signal.copy()
    #cut_f_signal[(freqs<6) & (freqs>-6)] = 0
    data = irfft(cut_f_signal)
    rms = np.sqrt(np.mean(np.array(data)**2))
    kurt=stats.kurtosis(data)
    if rms>.02 and kurt<300 and freqs_side[-1]>1550: ## to keep the range at least 800 hz
        FFT = scipy.fft.fft(data)
        FFT_side = FFT[range(N//2)] # one side FFT range, because we need just the haf of the frequency data because of symmetry
        abs_fft_1=abs(FFT_side)
        
        payload={
                        "Data_pull_time" : row["Data_pull_time"],
                        "Mac_Id" : row["Mac_Id"],
                        "Axis" : row["Axis"],
                        "Defect_Introduced" : row["Defect_Introduced"],
                        "Asset_Id" : row["Asset_Id"],
                        "Sensor_Loc":row["Sensor_Loc"],
                        "peak-peak": (max(data)-min(data)),
                        "rms": rms,
                        "kurt":kurt,
                        "crest":np.max(np.abs(data))/np.sqrt(np.mean(np.square(data))),
                        "variance":np.var(data),
                        "mean":np.mean(data),
                        "skewness":stats.skew(data),
                        'freqAmpliMean':np.mean(abs_fft_1),
                        'freqAmpliMax':max(abs_fft_1),
                        'freqAmpliKurt':stats.kurtosis(abs_fft_1),
                        'freqAmpliskw':stats.skew(abs_fft_1),
                        'freqAmpliVar':np.var(abs_fft_1),
                        }
        lenFreq=len(freqs_side)
        lastFreq=freqs_side[-1]
        abs_fft_1=list(abs_fft_1)
        jj=0
        feqv=1
        while jj < lenFreq and feqv<=8: ## for 8 bins from 0 to 1600 Hz
            #print('Condition satisfied')
            kk=int(round(jj+lenFreq/lastFreq*200)) ## 100 is freqency step size for binnning
            #print(kk)
            fft_Range=abs_fft_1[jj:kk]
            freq_sideRange=freqs_side[jj:kk]
            jj=kk
            payload['max_{}'.format(feqv)]=max(fft_Range)
            payload['mean_{}'.format(feqv)]=np.mean(fft_Range)
            payload['var_{}'.format(feqv)]=np.var(fft_Range)
            feqv+=1
        Data_Processed = Data_Processed.append(payload, ignore_index=True)



#for i,row in Data.iterrows():

In [ ]:
        lenFreq=len(freqs_side)
        lastFreq=freqs_side[-1]
        abs_fft_1=list(abs_fft_1)

In [ ]:
lenFreq

In [ ]:
raw = Data.iloc[0]["Vsample1":"Vsample4096" ]
raw_mean = Data.iloc[0]["Vsample1":"Vsample4096" ] - Data.iloc[0]["Vsample1":"Vsample4096" ].mean()

In [ ]:
Data_Processed.to_excel('Processed-sample.xlsx')

In [ ]:
stats.skew(raw_mean)

In [ ]:
plt.plot(raw_mean)

In [ ]:
### convert raw data to FFT

FFT = scipy.fft.fft(raw_mean)
plt.plot(abs(FFT[range(75,N//2)]) )

In [ ]:
freqs = scipy.fftpack.fftfreq(N, t[1]-t[0])

In [ ]:
max(abs(FFT))

In [ ]:
fft_data.to_excel('FFT_Data.xlsx')

In [ ]:
fft_data = pd.DataFrame()

In [ ]:
fft_data['Raw_data'] = raw_mean

In [ ]:
fft_data['FFT'] = FFT

In [ ]:
fft_data['abs_data'] = abs(FFT)

In [ ]:
f_signal = rfft(raw_mean)
fft_data['Rfft'] = f_signal

In [ ]:
len(f_signal)
#len((freqs<6) & (freqs>-6))
f_signal[(freqs<6) & (freqs>-6)]
f_signal.sum()

In [ ]:
f_signal[(freqs<6) & (freqs>-6)] = 0
fft_data['Rfft_6Freq'] = f_signal

In [ ]:
f_signal.sum()

In [ ]:
Data.loc[0:0,'Vsample1':'Vsample4096']

In [ ]:
row = Data_final.iloc[228]['Vsample1':'Vsample4096']
N=4096
sec = 1.279374957
fs_rate=round(N/sec) ## alternatively that means frequency

   
secs = N / float(fs_rate) ## time for which it will be played in sec

Ts = 1.0/fs_rate # sampling interval in time

t = np.arange(0, secs, Ts) # time vector as scipy arange field / numpy.ndarray

freqs = scipy.fft.fftfreq(N, t[1]-t[0])
print(freqs[4095])
freqs_side = freqs[range(N//2)] # one side frequency range
               
                ## caliberate the sensor in case there is gravitational effect left from the sensor settings
data = Data.iloc[4]['Vsample1':'Vsample4096']
meanVal=np.mean(data)
data-=meanVal
   
FFT = scipy.fft.fft(raw_mean)
FFT_side = FFT[range(N//2)] # one side FFT range, because we need just the haf of the frequency data because of symmetry
abs_fft_1=2*abs(FFT_side)/N


type(abs_fft_1)
#plt.plot(abs(FFT[range(N//2)]))
plt.plot(FFT)

1/t[1]

## K means clustering

In [ ]:
from sklearn.cluster import KMeans
model = KMeans(n_clusters=2)

In [ ]:
model.fit(processed[["P2P","RMS","Mean","Max"]])

In [ ]:
labels = model.predict(processed[["P2P","RMS","Mean","Max"]])

In [ ]:
print(labels)

### understanding our cluster

In [ ]:
colors = ["red", "orange", "green", "blue", "purple", "gray"]
Fault = ["Hanoi", "Nha Trang", "Vung Tau", "Phu Quoc", "Quang Ngai", "Saigon"]
region_colors=dict(zip(regions,colors))

grp_color=[]
for i in data['Region']:
    grp_color.append(region_colors[i]) 

x_long=data[' Longitude']
y_lat=data[" Latitude"]
plt.scatter(x_long,y_lat,c=grp_color)
plt.legend(grp_color,regions,loc='right')

In [ ]:

plt.scatter(processed.P2P,processed.RMS, c=processed["Actual_Cluster"].apply(lambda x: colors[x]),label=)
plt.show()
plt.legend()
plt.show()

In [ ]:
centroids = model.cluster_centers_
centroids_x = centroids[:,0]
centroids_y = centroids[:,1]

In [ ]:
plt.scatter(processed.P2P,processed.RMS, c=labels)
plt.scatter(centroids_x, centroids_y, marker='X', s=200)
plt.show()

In [ ]:
Load_NoLoad_group = {"No Defect_No Load": "No_Load",
                     "No Defect_Empty Tray":"Load",
                     "No Defect_with no tray" : "No_Load" }

processed["Actual_Cluster"] = [ Load_NoLoad_group[i] for i in processed.Defect_Introduced]

In [ ]:
processed["Cluster"] = labels

In [ ]:
processed[["Defect_Introduced","Actual_Cluster","Cluster"]].to_excel("Cluster_data_2.xlsx")

In [ ]:
plt.scatter(processed.P2P,processed.RMS, c=processed["Cluster"])
plt.show()

In [ ]:
fig, ax = plt.subplots()

    ax.scatter(x, y, c=color, s=scale, label=color,
               alpha=0.3, edgecolors='none')

ax.legend()
ax.grid(True)

plt.show()

In [ ]:
#row = Data_final.iloc[228]['Vsample1':'Vsample4096']
N=4096
sec = 1.279374957
fs_rate=round(N/sec) ## alternatively that means frequency

   
secs = N / float(fs_rate) ## time for which it will be played in sec

Ts = 1.0/fs_rate # sampling interval in time

t = np.arange(0, secs, Ts) # time vector as scipy arange field / numpy.ndarray

freqs = scipy.fft.fftfreq(N, t[1]-t[0])
#print(freqs[4095])
freqs_side = freqs[range(N//2)] # one side frequency range

for j in ['X','Y','Z']:
    print(j)
    for i,row in Data_final[(Data_final.Defect_Introduced == "No Defect_Empty Tray")  & (Data_final.Asset_Id == "101.CA.DTL.01.1-030") & (Data_final.Axis == j)].head().iterrows():
    #if (row['Vsample1':'Vsample4096'].max() > 2) and (row['Vsample1':'Vsample4096'].min() < 0) :
        data = row['Vsample1':'Vsample4096']
        fig, axs = plt.subplots(2)
        axs[0].plot(data)
        axs[0].set_ylim(-4,4)
        fig.suptitle(str(row['Data_pull_time']) +'\n'+row['Defect_Introduced']+'\n'+row['Sensor_Loc'])
        #fig.title(str(row['Data_pull_time'])+' '+row['Axis'])
        
        FFT = scipy.fft.fft(data)
        axs[1].plot(FFT)
        fig.savefig('./101.CA.DTL.01.1-030/Load/'+j+'/'+row['Mac_Id'].replace(":","")+'_'+row['Axis']+'_'+str(i)+'.png',dpi=400)
        #fig.savefig('test.png')
        plt.close()

In [ ]:
for j in ['X','Y','Z']:
    for i,row in Data_final[(Data_final.Defect_Introduced == "No Defect_No Load")  & (Data_final.Asset_Id == "101.CA.DTL.01.1-050") & (Data_final.Axis == j)].iterrows():
        #if (row['Vsample1':'Vsample4096'].max() > 2) and (row['Vsample1':'Vsample4096'].min() < 0) :
        plot = row['Vsample1':'Vsample4096'].plot()
        plot.set_ylim(-4,4)
        fig = plot.get_figure()
        plt.suptitle(str(row['Data_pull_time'])+' '+row['Axis'])
        plt.title(row['Defect_Introduced']+' '+row['Sensor_Loc'])
        fig.savefig('./101.CA.DTL.01.1-030/Load/X/'+row['Mac_Id'].replace(":","")+'_'+row['Axis']+'_'+str(i)+'.png')
        plt.close()

In [ ]:
fig, ax = plt.subplots(2, 
           sharex='col', sharey='row')

ax[0].plot(row)

ax[1].plot([3,4,5])

plt.show()

In [ ]:
row = Data_final.iloc[0]['Vsample1':'Vsample4096']
Data_final.iloc[0]

### clustering part 2 

In [ ]:
Data_Processed.head()

points = Data_Processed.loc[:,["crest","freqAmpliKurt"]].values

In [ ]:
model = KMeans(n_clusters=2)
model.fit(points)
labels = model.predict(points)
xs = points[:,0]
ys = points[:,1]
plt.scatter(xs, ys, c=labels)
plt.show()

In [ ]:
from sklearn.cluster import KMeans

model = KMeans(n_clusters=2)
model.fit(points)
labels = model.predict(points)

In [ ]:
for i in Data_Processed.columns[6:]:
    for j in Data_Processed.columns[6:]:
        if i != j:
            print(i,j)
            points = Data_Processed.loc[:,[i,j]].values
            model = KMeans(n_clusters=2)
            model.fit(points)
            labels = model.predict(points)
            xs = points[:,0]
            ys = points[:,1]
            fig = plt.figure()
            plt.scatter(xs, ys, c=labels)
            plt.xlabel(i)
            plt.ylabel(j)
            plt.show()
            fig.savefig('./Cluster/'+i+'-Vs-'+j+'.png',dpi=400)


## PCA

In [ ]:

Load_NoLoad_group = {"No Defect_No Load": "No_Load",
                     "No Defect_Empty Tray":"Load",
                     "No Defect_with no tray" : "No_Load" }

Data_Processed["Actual_Cluster"] = [ Load_NoLoad_group[i] for i in Data_Processed.Defect_Introduced]

In [ ]:
Data_Processed.head()

In [ ]:
Data_Processed.columns

Data_rearrange = Data_Processed[['Asset_Id', 'Axis', 'Data_pull_time', 'Defect_Introduced', 'Mac_Id',
       'Sensor_Loc','Actual_Cluster', 'crest', 'freqAmpliKurt', 'freqAmpliMax', 'freqAmpliMean',
       'freqAmpliVar', 'freqAmpliskw', 'kurt', 'max_1', 'max_2', 'max_3',
       'max_4', 'max_5', 'max_6', 'max_7', 'max_8', 'mean', 'mean_1', 'mean_2',
       'mean_3', 'mean_4', 'mean_5', 'mean_6', 'mean_7', 'mean_8', 'peak-peak',
       'rms', 'skewness', 'var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'var_6',
       'var_7', 'var_8', 'variance']]

In [ ]:
Data_rearrange.head()

In [ ]:
EDTgrouped=Data_rearrange.loc[:,"Defect_Introduced":"variance"]
EDTgrouped=EDTgrouped.drop(columns=['Mac_Id','Sensor_Loc'])
EDTgrouped.head()

In [ ]:
X = EDTgrouped.loc[:,"crest":]  # independent variables data
y = EDTgrouped.Actual_Cluster  # dependednt variable data
EDTgrouped.head()

In [ ]:
X.apply(np.mean)
# Std Deviation
X.apply(np.std)

In [ ]:
#Mean and Std deviation by Gp
def printMeanAndSdByGroup(variables, groupvariable):
    data_groupby = variables.groupby(groupvariable)
    print("## Means:")
    display(data_groupby.apply(np.mean))
    print("\n## Standard deviations:")
    display(data_groupby.apply(np.std))
    print("\n## Sample sizes:")
    display(pd.DataFrame(data_groupby.apply(len)))

In [ ]:
printMeanAndSdByGroup(X, y)

### Standardisation

In [ ]:
standardisedX = scale(X)
standardisedX = pd.DataFrame(standardisedX, index=X.index, columns=X.columns)

In [ ]:
standardisedX.apply(np.mean)

In [ ]:
standardisedX.apply(np.std)

In [ ]:
pca = PCA().fit(standardisedX)

In [ ]:
type(pca)

In [ ]:
def pca_summary(pca, standardised_data, out=True):
    names = ["PC"+str(i) for i in range(1, len(pca.explained_variance_ratio_)+1)]
    a = list(np.std(pca.transform(standardised_data), axis=0))
    b = list(pca.explained_variance_ratio_)
    c = [np.sum(pca.explained_variance_ratio_[:i]) for i in range(1, len(pca.explained_variance_ratio_)+1)]
    columns = pd.MultiIndex.from_tuples([("sdev", "Standard deviation"), ("varprop", "Proportion of Variance"), ("cumprop", "Cumulative Proportion")])
    summary = pd.DataFrame(zip(a, b, c), index=names, columns=columns)
    if out:
        print("Importance of components:")
        display(summary)
    return summary

In [ ]:
summary = pca_summary(pca, standardisedX)

In [ ]:
def screeplot(pca, standardised_values):
    y = np.std(pca.transform(standardised_values), axis=0)**2
    x = np.arange(len(y)) + 1
    plt.plot(x, y, "o-")
    plt.xticks(x, ["Comp."+str(i) for i in x], rotation=60)
    plt.ylabel("Variance")
    plt.show()

screeplot(pca, standardisedX)

In [ ]:
# Using Kaiser’s criterion: that we should only retain principal components for which the variance is above 1
summary.sdev**2

In [ ]:
def pca_scatter(pca, standardised_values, classifs):
    foo = pca.transform(standardised_values)
    bar = pd.DataFrame(zip(foo[:, 0], foo[:, 1], classifs), columns=["PC1", "PC2", "Class"])
    sns.lmplot("PC1", "PC2", bar, hue="Class", fit_reg=False)

pca_scatter(pca, standardisedX, y)